In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

from helper_functions import get_classification_report
from helper_functions import create_tf_data_datasets_contrastive
from helper_functions import create_tf_data_testset_contrastive
from helper_functions import euclidean_distance
from helper_functions import contrastive_loss
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras import mixed_precision

In [3]:
mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 3090, compute capability 8.6


In [6]:
def AlexNet(height, width, channels):

    X_input = keras.layers.Input(shape=(height, width, channels))

    X = keras.layers.Conv2D(96,(11,11),strides = 4,name="conv0")(X_input)
    X = keras.layers.BatchNormalization(axis = 3 , name = "bn0")(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.MaxPooling2D((3,3),strides = 2,name = 'max0')(X)

    X = keras.layers.Conv2D(256,(5,5),padding = 'same' , name = 'conv1')(X)
    X = keras.layers.BatchNormalization(axis = 3 ,name='bn1')(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.MaxPooling2D((3,3),strides = 2,name = 'max1')(X)

    X = keras.layers.Conv2D(384, (3,3) , padding = 'same' , name='conv2')(X)
    X = keras.layers.BatchNormalization(axis = 3, name = 'bn2')(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.Conv2D(384, (3,3) , padding = 'same' , name='conv3')(X)
    X = keras.layers.BatchNormalization(axis = 3, name = 'bn3')(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.Conv2D(256, (3,3) , padding = 'same' , name='conv4')(X)
    X = keras.layers.BatchNormalization(axis = 3, name = 'bn4')(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.MaxPooling2D((3,3),strides = 2,name = 'max2')(X)

    X = keras.layers.Flatten()(X)

    X = keras.layers.Dense(4096, activation = 'relu', name = "fc0")(X)

    X = keras.layers.Dense(4096, activation = 'sigmoid', name = 'fc1')(X)

    model = keras.models.Model(inputs = X_input, outputs = X, name='AlexNet')

    return model

In [7]:
def half_deep_writer(height, width, channels):

    input = keras.layers.Input(shape=(height, width, channels))

    x = keras.layers.Conv2D(96, kernel_size=5, strides=2, activation='relu')(input)
    x = keras.layers.MaxPooling2D(3, strides=2)(x)

    x = keras.layers.Conv2D(256, kernel_size=3, strides=1, padding="valid", activation='relu')(x)
    x = keras.layers.MaxPooling2D(3, strides=2)(x)

    x = keras.layers.Conv2D(384, kernel_size=3, strides=1, padding="valid", activation='relu')(x)
    x = keras.layers.Conv2D(384, kernel_size=3, strides=1, padding="valid", activation='relu')(x)
    x = keras.layers.Conv2D(256, kernel_size=3, strides=1, padding="valid",  activation='relu')(x)
    x = keras.layers.MaxPooling2D(3, strides=2)(x)

    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(1024, activation='relu')(x)
    output = keras.layers.Dense(1024, activation='relu')(x)

    model = keras.models.Model(input, output)

    return model

# Alexnet - 90K - RGB - Contrastive - Eucledian

In [6]:
anchor_images_path = "npz_datasets/pairs_90k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_90k_224_224/positive"
width, height, channels = 224, 224, 3
batch_size = 256
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

anchor_images_test_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_test_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=True)

In [7]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "Adam",
                         "loss_function": "contrastive loss",
                         "epochs": 50,
                         "architecture": "Alexnet - 90k - RGB"})

config = wandb.config

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [7]:
model = AlexNet(height, width, channels)

left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = keras.layers.Lambda(euclidean_distance)([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/50
286/286 [==============================] - 69s 180ms/step - loss: 0.2205 - accuracy: 0.6626 - val_loss: 0.2291 - val_accuracy: 0.6202
Epoch 2/50
286/286 [==============================] - 55s 176ms/step - loss: 0.2128 - accuracy: 0.6694 - val_loss: 0.2588 - val_accuracy: 0.5252
Epoch 3/50
286/286 [==============================] - 55s 176ms/step - loss: 0.2090 - accuracy: 0.6741 - val_loss: 0.2761 - val_accuracy: 0.6213
Epoch 4/50
286/286 [==============================] - 55s 176ms/step - loss: 0.2152 - accuracy: 0.6686 - val_loss: 0.2653 - val_accuracy: 0.4989
Epoch 5/50
286/286 [==============================] - 55s 176ms/step - loss: 0.2157 - accuracy: 0.6663 - val_loss: 0.2749 - val_accuracy: 0.5853
Epoch 6/50
286/286 [==============================] - 55s 176ms/step - loss: 0.2092 - accuracy: 0.6758 - val_loss: 0.2726 - val_accuracy: 0.5025
Epoch 7/50
286/286 [==============================] - 55s 176ms/step - loss: 0.2087 - accuracy: 0.6708 - val_loss: 0.2078 - val_ac

In [10]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

# Alexnet - 90K - RGB - Crossentropy - Eucledian

In [13]:
anchor_images_path = "npz_datasets/pairs_90k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_90k_224_224/positive"
width, height, channels = 224, 224, 3
batch_size = 256
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

anchor_images_test_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_test_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=True)

In [14]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "Adam",
                         "loss_function": "Binary Crossentropy",
                         "epochs": 50,
                         "architecture": "Alexnet - 90k - RGB"})

config = wandb.config

In [15]:
model = AlexNet(height, width, channels)

left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = keras.layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = keras.layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/50
286/286 [==============================] - 64s 178ms/step - loss: 0.6133 - accuracy: 0.6622 - val_loss: 0.7534 - val_accuracy: 0.6017
Epoch 2/50
286/286 [==============================] - 54s 173ms/step - loss: 0.5912 - accuracy: 0.6770 - val_loss: 0.6438 - val_accuracy: 0.6420
Epoch 3/50
286/286 [==============================] - 54s 173ms/step - loss: 0.5834 - accuracy: 0.6811 - val_loss: 0.7140 - val_accuracy: 0.6462
Epoch 4/50
286/286 [==============================] - 55s 174ms/step - loss: 0.5781 - accuracy: 0.6823 - val_loss: 0.5918 - val_accuracy: 0.6795
Epoch 5/50
286/286 [==============================] - 54s 173ms/step - loss: 0.5737 - accuracy: 0.6861 - val_loss: 0.6932 - val_accuracy: 0.5934
Epoch 6/50
286/286 [==============================] - 55s 173ms/step - loss: 0.5690 - accuracy: 0.6855 - val_loss: 0.5794 - val_accuracy: 0.6777
Epoch 7/50
286/286 [==============================] - 54s 173ms/step - loss: 0.5711 - accuracy: 0.6849 - val_loss: 0.7822 - val_ac

In [16]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

# Half DeepWriter - 90K - RGB - Contrastive - Eucledian

In [6]:
anchor_images_path = "npz_datasets/pairs_90k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_90k_224_224/positive"
width, height, channels = 224, 224, 3
batch_size = 256
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

anchor_images_test_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_test_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=True)

In [7]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "Adam",
                         "loss_function": "contrastive loss",
                         "epochs": 50,
                         "architecture": "Half Deep Writer - 90k - RGB"})

config = wandb.config

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [8]:
model = half_deep_writer(height, width, channels)

left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = keras.layers.Lambda(euclidean_distance)([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/50
286/286 [==============================] - 95s 255ms/step - loss: 0.2202 - accuracy: 0.5964 - val_loss: 0.2030 - val_accuracy: 0.6824
Epoch 2/50
286/286 [==============================] - 74s 242ms/step - loss: 0.1987 - accuracy: 0.7048 - val_loss: 0.1860 - val_accuracy: 0.7404
Epoch 3/50
286/286 [==============================] - 75s 244ms/step - loss: 0.1810 - accuracy: 0.7480 - val_loss: 0.1754 - val_accuracy: 0.7636
Epoch 4/50
286/286 [==============================] - 74s 243ms/step - loss: 0.1695 - accuracy: 0.7690 - val_loss: 0.1634 - val_accuracy: 0.7850
Epoch 5/50
286/286 [==============================] - 74s 243ms/step - loss: 0.1624 - accuracy: 0.7792 - val_loss: 0.1557 - val_accuracy: 0.7933
Epoch 6/50
286/286 [==============================] - 74s 243ms/step - loss: 0.1552 - accuracy: 0.7925 - val_loss: 0.1533 - val_accuracy: 0.7907
Epoch 7/50
286/286 [==============================] - 75s 244ms/step - loss: 0.1479 - accuracy: 0.8036 - val_loss: 0.1399 - val_ac

In [9]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

# Half DeepWriter - 90K - RGB - Crossentropy - Eucledian

In [8]:
anchor_images_path = "npz_datasets/pairs_90k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_90k_224_224/positive"
width, height, channels = 224, 224, 3
batch_size = 256
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

anchor_images_test_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_test_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=True)

In [9]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "Adam",
                         "loss_function": "Binary Crossentropy",
                         "epochs": 50,
                         "architecture": "Half Deep Writer - 90k - RGB"})

config = wandb.config

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [10]:
model = half_deep_writer(height, width, channels)

left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = keras.layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = keras.layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
siamese_model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/50
286/286 [==============================] - 96s 257ms/step - loss: 0.6322 - accuracy: 0.5819 - val_loss: 0.6022 - val_accuracy: 0.6479
Epoch 2/50
286/286 [==============================] - 75s 244ms/step - loss: 0.5784 - accuracy: 0.6896 - val_loss: 0.5426 - val_accuracy: 0.7005
Epoch 3/50
286/286 [==============================] - 74s 243ms/step - loss: 0.5210 - accuracy: 0.7400 - val_loss: 0.5009 - val_accuracy: 0.7491
Epoch 4/50
286/286 [==============================] - 75s 244ms/step - loss: 0.4956 - accuracy: 0.7622 - val_loss: 0.4810 - val_accuracy: 0.7766
Epoch 5/50
286/286 [==============================] - 75s 244ms/step - loss: 0.4805 - accuracy: 0.7714 - val_loss: 0.4682 - val_accuracy: 0.7868
Epoch 6/50
286/286 [==============================] - 74s 243ms/step - loss: 0.4749 - accuracy: 0.7766 - val_loss: 0.4539 - val_accuracy: 0.7866
Epoch 7/50
286/286 [==============================] - 75s 243ms/step - loss: 0.4490 - accuracy: 0.7979 - val_loss: 0.4356 - val_ac

In [11]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, siamese_model)
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()